# 情報システム論実習課題 テキスト分析

## 利用する文書
様々な国のWikipediaにおけるabstractを取り出したデータセット．

## 利用する表現手法

ここでは利用する表現手法として，BoWとTF-IDFを利用する．

## 利用する距離
BoWについては，マンハッタン距離とユークリッド距離，コサイン類似度によって計算する．
TF-IDFについては，コサイン類似度を計算する．

## 実際の処理

### 必要なパッケージのインストール

In [1]:
!pip install -q nltk
!pip install -q gensim

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 必要なパッケージのインポート・ダウンロード

In [2]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/restartsugar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/restartsugar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/restartsugar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### データのデータフレームへの格納

In [3]:
df = pd.read_csv("./nlp_country.csv")
df

,Name,Abstract
0,Japan,Japan is an island country in East Asia. Locat...
1,United States,"The United States of America (USA), commonly k..."
2,England,England is a country that is part of the Unite...
3,China,"China, officially the People's Republic of Chi..."
4,India,"India, also known as the Republic of India,[19..."
5,Korea,Korea is a region in East Asia.[3] Since 1948 ...
6,Germany,"Germany, officially the Federal Republic of Ge..."
7,Russia,"Russia, or the Russian Federation[12], is a tr..."
8,France,"France, officially the French Republic, is a c..."
9,Italy,"Italy, officially the Italian Republic,[10][11..."


### 前処理

stop wordsで冠詞、代名詞、前置詞など一般的な語を削除するための関数を定義する．今回は英語であるので英語のstop wordsのリストを利用する．

In [4]:
en_stop = nltk.corpus.stopwords.words('english')
def remove_stopwords(word, stopwordset):
  if word in stopwordset:
    return None
  else:
    return word

具体的に，前処理をしていく．

In [5]:
def preprocessing_text(text):
  def cleaning_text(text):
    # @の削除
    pattern1 = '@|%'
    text = re.sub(pattern1, '', text)    
    pattern2 = '\[[0-9 ]*\]'
    text = re.sub(pattern2, '', text)    
    # <b>タグの削除
    pattern3 = '\([a-z ]*\)'
    text = re.sub(pattern3, '', text)    
    pattern4 = '[0-9]'
    text = re.sub(pattern4, '', text)
    return text
  
  def tokenize_text(text):
    text = re.sub('[.,]', '', text)
    return text.split()

  def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma
    
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens
  
docs = df["Abstract"].values
pp_docs = [preprocessing_text(text) for text in docs]

### 前処理したドキュメントで計算
前処理したdocumentを用いて，BoWとTF-IDFを計算する．

#### Cosine類似度を計算する関数の定義

In [6]:
def cosine_similarity(list_a, list_b):
  inner_prod = np.array(list_a).dot(np.array(list_b))
  norm_a = np.linalg.norm(list_a)
  norm_b = np.linalg.norm(list_b)
  try:
      return inner_prod / (norm_a*norm_b)
  except ZeroDivisionError:
      return 1.0

#### ミンコフスキー距離を計算する関数の定義

In [7]:
def minkowski_distance(list_a, list_b, p):
  diff_vec = np.array(list_a) - np.array(list_b)
  return np.linalg.norm(x=diff_vec, ord=p)

### Bag of Words
まずはBag of Wordsから計算する．bowのベクトルを計算する関数を定義する．

In [8]:
def bow_vectorizer(docs):
  word2id = {}
  for doc in docs:
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
        
  result_list = []
  for doc in docs:
    doc_vec = [0] * len(word2id)
    for w in doc:
      doc_vec[word2id[w]] += 1
    result_list.append(doc_vec)
  return result_list, word2id

実際に計算する．

In [9]:
bow_vec, word2id = bow_vectorizer(pp_docs)

今回は日本のドキュメントと最もよく似ている国を示す．まず，コサイン類似度による計算を行う．

In [10]:
def calc_cosine(vector, vector_list):
  result = {}
  for i, x in enumerate(vector_list):
    result[i] = cosine_similarity(vector, vector_list[i])
    
  return result

print("BoW: cosine類似度")
res = calc_cosine(bow_vec[0],bow_vec)
res

BoW: cosine類似度


{0: 1.0000000000000002,
 1: 0.30868394472237093,
 2: 0.24442109959824154,
 3: 0.31380271433857343,
 4: 0.24479467332586202,
 5: 0.34372825633151943,
 6: 0.3234461542498459,
 7: 0.30413196719287744,
 8: 0.33713265112489266,
 9: 0.3270549101118192,
 10: 0.2846409928828126,
 11: 0.22429633498453364,
 12: 0.2846361894175649,
 13: 0.3037161720774084,
 14: 0.3871304096992261,
 15: 0.3196186379883876}

結果から日本を除いて最も数値が大きいのは14の国であり，その国はインドネシアである．

次に日本のドキュメントと各ドキュメントのミンコフスキー距離を計算するための関数を示す．
今回はミンコフスキー距離の$p$は，$p=1$， $p=2$の2通りで計算する．

In [11]:
def calc_minkowski(vector, vector_list, p):
    result = {}
    result = {}
    for i, x in enumerate(vector_list):
        result[i] = minkowski_distance(vector, x, p)
    
    return result

マンハッタン距離の場合($p=1$)を示す．

In [12]:
print("BoW: マンハッタン距離")
res = ｃalc_minkowski(bow_vec[0],bow_vec, 1)
res   

BoW: マンハッタン距離


{0: 0.0,
 1: 595.0,
 2: 474.0,
 3: 503.0,
 4: 467.0,
 5: 528.0,
 6: 512.0,
 7: 614.0,
 8: 522.0,
 9: 597.0,
 10: 496.0,
 11: 513.0,
 12: 579.0,
 13: 478.0,
 14: 505.0,
 15: 504.0}

日本を除くマンハッタン距離が最も小さいのは4でありその国はインドである．

次にユークリッド距離の場合を示す．

In [13]:
print("BoW: ユークリッド距離")
res = calc_minkowski(bow_vec[0],bow_vec, 2)
res   

BoW: ユークリッド距離


{0: 0.0,
 1: 44.05678154382138,
 2: 34.87119154832539,
 3: 34.0147027033899,
 4: 32.202484376209235,
 5: 35.07135583350036,
 6: 35.608987629529715,
 7: 37.815340802378074,
 8: 35.66510900025401,
 9: 39.102429592034305,
 10: 34.438350715445125,
 11: 33.8673884437522,
 12: 38.3275357934736,
 13: 32.341923257592455,
 14: 31.76476034853718,
 15: 34.785054261852174}

日本を除くユークリッド距離が最も小さいのは4でありその国はインドである．

### 考察

結果から，ユークリッド距離とcosine類似度の結果が一致しており，マンハッタン距離が異なるということが分かる．
これはマンハッタン距離は単純な出現文字数の差を取る一方で，その二乗和をとってルートを取るという点で異なることからこのような結果の違いとして現れたと考える．また，cosine類似度はそのベクトルの角度が近いすなわち，文書の内容の方向性が近いものが抽出できる．こうした計算方法の違いで実際に近い文書として出力されることが結果からわかった．

### TF-IDF
つぎにTF-IDFを計算する．TF-IDFのベクトルを計算する関数を定義する．

In [14]:
def tfidf_vectorizer(docs):
  def tf(word2id, doc):
    term_counts = np.zeros(len(word2id))
    for term in word2id.keys():
      term_counts[word2id[term]] = doc.count(term)
    tf_values = list(map(lambda x: x/sum(term_counts), term_counts))
    return tf_values
  
  def idf(word2id, docs):
    idf = np.zeros(len(word2id))
    for term in word2id.keys():
      idf[word2id[term]] = np.log(len(docs) / sum([bool(term in doc) for doc in docs]))
    return idf
  
  word2id = {}
  for doc in docs:
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
  
  return [[_tf*_idf for _tf, _idf in zip(tf(word2id, doc), idf(word2id, docs))] for doc in docs], word2id

実際に前処理をした文書で計算する．

In [15]:
tfidf_vec, word2id = tfidf_vectorizer(pp_docs)

BoWと同様，日本の文書と似ている文書を見つける．まずは，cosine類似度の計算をする．

In [18]:
print("TF-IDF: cosine類似度")
res = calc_cosine(tfidf_vec[0],tfidf_vec)
res

TF-IDF: cosine類似度


{0: 1.0,
 1: 0.04945156965230686,
 2: 0.0355002685981015,
 3: 0.07494324927746153,
 4: 0.02200165046387345,
 5: 0.089213868005443,
 6: 0.04329186935344453,
 7: 0.04340970910393382,
 8: 0.05061679443369346,
 9: 0.05446867547327852,
 10: 0.03479541972998953,
 11: 0.03392463518350004,
 12: 0.038469390607195876,
 13: 0.05035814117836253,
 14: 0.06794378321355647,
 15: 0.029516361108928312}

結果から，最も近い文書は5の韓国であることが分かる．  
次に，マンハッタン距離を求める．

In [19]:
print("TF-IDF: マンハッタン距離")
res = ｃalc_minkowski(tfidf_vec[0],tfidf_vec, 1)
res   

TF-IDF: マンハッタン距離


{0: 0.0,
 1: 2.3608735368449936,
 2: 2.5731221468801357,
 3: 2.6294207675543775,
 4: 2.7744931615739383,
 5: 2.6773565232088843,
 6: 2.43907571391646,
 7: 2.653363612428077,
 8: 2.318776070485786,
 9: 2.5387188371556677,
 10: 2.45044237565666,
 11: 2.5885343221688917,
 12: 2.571199454754217,
 13: 2.58376708137673,
 14: 2.564644164768093,
 15: 2.489820098650106}

マンハッタン距離が最も小さいのは8のフランスであることが分かる．  
次に，ユークリッド距離を求める．

In [21]:
print("TF-IDF: ユークリッド距離")
res = ｃalc_minkowski(tfidf_vec[0],tfidf_vec, 2)
res   

TF-IDF: ユークリッド距離


{0: 0.0,
 1: 0.13894063261041328,
 2: 0.1954296993588011,
 3: 0.16682345000092424,
 4: 0.18428766675062896,
 5: 0.18393038894598424,
 6: 0.17064933882760383,
 7: 0.1591377644349097,
 8: 0.1489531710857635,
 9: 0.16479906407130404,
 10: 0.16829228129212442,
 11: 0.1646238972305546,
 12: 0.15311773054039643,
 13: 0.1719659245803892,
 14: 0.1530401348289774,
 15: 0.16792101813795798}

ユークリッド距離が最も小さいのは1のアメリカであることが分かる．  

### 考察
TF-IDFの場合，結果が全ての距離尺度で異なる結果となった．また，BoWの結果と比較しても各尺度で一致している結果はなく，TF-IDFとBoWでは異なるアルゴリズムの結果が現れていると考えられる．一般的に用いられている尺度であるcosine類似度は韓国の文書が最も似ているという結果となった．TF-IDFは単語の出現頻度と，逆文書頻度を利用するため，単純な出現頻度を見る，BoWとは結果が異なると考える．また，利用する尺度や表現手法によってこうした結果の違いとして現れるため，利用する尺度は慎重に選定する必要があると考える．

## 感想
利用する尺度，表現手法によって違いが実際に現れることが分かった．また，TF-IDFやBoWはコードを見ると案外簡単にかけることがわかった．一方難しいと感じたのは，前処理の仕方であり，この部分がしっかりしていないと精度の高い文書比較は出来なさそうであると感じた．